In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import zipfile
import os
from IPython.display import FileLink
from PIL import Image
from torch.utils.data import Dataset
import torch.nn as nn
import torch
from torchinfo import summary
from torchvision.utils import save_image

from torch.utils.data import  DataLoader
from torchvision import transforms
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
import torch.nn.functional as F
import torch.optim as optim
import os


os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#          print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

ModuleNotFoundError: No module named 'pandas'

In [ ]:
folder_dir_in_matthijs = r'C:\Users\matth\Documents\Master Nanobiology\Deep learning\project\Images input'
folder_dir_gt_matthijs = r"C:\Users\matth\Documents\Master Nanobiology\Deep learning\project\Images GT"
folder_dir_en_matthijs = r"C:\Users\matth\Documents\Master Nanobiology\Deep learning\project\Images G"

In [ ]:
class Discriminator(nn.Module):
    
    def __init__(self, latent_dims, s_img, hdim, kernel_size=(4, 4), stride=2):
        
        super(Discriminator, self).__init__()

        ########################################################################
        #    Create the necessary layers                                 #
        ########################################################################

        self.layers = nn.ModuleList()
        
        # Input layer dim -- down1
        self.layers.append(nn.Conv2d(in_channels=6, out_channels=64, kernel_size=kernel_size, stride=2, padding=1))

        # Hidden to hidden convolution -- down2 and down 3
        for i in range(0, 2):
            self.layers.append(nn.Conv2d(in_channels=hdim[i],
                                             out_channels=hdim[i + 1],
                                             kernel_size=kernel_size, stride = stride, padding=1))

        # Pad with zeroes
        self.layers.append(nn.ZeroPad2d(padding=(1,1,1,1)))

        # Conv2D
        self.layers.append(nn.Conv2d(in_channels=hdim[3],
                                             out_channels=hdim[4],
                                             kernel_size=kernel_size, stride = 1))

        # Batchnorm
        self.layers.append(nn.BatchNorm2d(hdim[4]))

        # Zeropad2
        self.layers.append(nn.ZeroPad2d(padding=(1,1,1,1)))

        #Conv2D 2
        self.layers.append(nn.Conv2d(in_channels=hdim[5],
                                             out_channels=hdim[6],
                                             kernel_size=kernel_size, stride = 1))

        self.Leakyrelu = nn.LeakyReLU(0.2)
        

    def forward(self, x):

        for n_layer, layer in enumerate(self.layers):
            ## The fourth layer first has a batchnorm and then a Leakyrelu
            if n_layer != 4:
                x = self.Leakyrelu(layer(x))
            else:
                x = layer(x)
        return x

In [ ]:
def train_discriminator():

    folder_dir_gt = folder_dir_gt_matthijs
    folder_dir_in = folder_dir_in_matthijs
    folder_dir_en = folder_dir_en_matthijs

    in_list = [os.path.join(folder_dir_in, filename) for filename in os.listdir(folder_dir_en)]
    gt_list = [os.path.join(folder_dir_gt, filename) for filename in os.listdir(folder_dir_gt)]
    enhanced_list = [os.path.join(folder_dir_en, filename) for filename in os.listdir(folder_dir_in)]

    # =============================================================================
    # test auo-encoder
    # =============================================================================
    n_samples, in_channels, s_img, latent_dims = len(in_list) * 2, 6, 256, 512 # 6 for two images
    hdim = [64, 128, 256, 256, 512, 512, 1] #choose hidden dimension discriminator
    output_shape = (n_samples, 1, 30, 30)

    # =============================================================================
    # Fill x with stacks of fake images (random) and real images (all ones)
    # =============================================================================
    # real_image = torch.ones(in_channels//2, s_img, s_img)

    transform = transforms.Compose([
        transforms.Resize((256, 256), transforms.InterpolationMode.NEAREST),  # Resize the images to a consistent size
        # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize the images
        transforms.ToTensor()           # Convert the images to PyTorch tensors
    ])


    training_list = []

    # create training list with right size (transformed)
    for i in range(len(in_list)):
        input_img = transform(Image.open(in_list[i]))
        enhanced_img = transform(Image.open(enhanced_list[i]))
        gt_img = transform(Image.open(gt_list[i]))

        # Normalize range [-1, 1]
        input_img = torch.tanh(input_img)
        enhanced_img = torch.tanh(enhanced_img)
        gt_img = torch.tanh(gt_img)

        # List image combinations with label
        training_list.append((input_img, enhanced_img, False))
        training_list.append((input_img, gt_img, True))

    random.shuffle(training_list)

    x = torch.empty((n_samples, in_channels, s_img, s_img))
    labels = torch.empty(output_shape)


    for i, (img1, img2, is_real) in enumerate(training_list):

        x[i, :, :, :] = torch.cat((img1, img2), dim=0)

        if is_real:
            labels[i, 0] = torch.ones(30, 30)
        else:
            labels[i, 0] = torch.zeros(30, 30)


    # =============================================================================
    # initialize model
    # =============================================================================
    model = Discriminator(latent_dims, s_img, hdim)
    summary(model, (in_channels, s_img, s_img), device='cpu') # (in_channels, height, width)

    # =============================================================================
    # training of the Discriminator
    # =============================================================================
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=2e-4)
    n_epochs = 2
    losses = []

    # Training loop
    for epoch in range(n_epochs):

        # Forward pass
        outputs = model(x)
        output_probabilities = torch.sigmoid(outputs)

        # Calculate loss
        loss = criterion(output_probabilities, labels)
        print(f'Epoch [{epoch + 1}/{n_epochs}], Loss: {loss.item()}')
        losses.append(loss.item())

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # =============================================================================
    # plot the training Loss over epochs (on its own data,
    # so will eventually convege to 0)
    # =============================================================================

    plt.plot(losses)
    plt.ylim(0, .8)
    plt.xlabel('epochs')
    plt.ylabel('Loss (BCE)')
    plt.show()

In [ ]:
def train_generator(input_imgs, real_imgs):

    n_samples, in_channels, s_img, latent_dims = len(input_imgs), 6, 256, 512 # 6 for two images
    hdim = [64, 128, 256, 256, 512, 512, 1] #choose hidden dimension discriminator
    x_generator = torch.empty((n_samples, in_channels, s_img, s_img))
    x_discriminator = torch.empty((n_samples, in_channels, s_img, s_img))

    generator = Generator() # TODO:
    discriminator  = Discriminator(latent_dims, s_img, hdim)
    loss_L1 = nn.L1Loss()
    loss_BCE = nn.BSELoss()
    optimizer = optim.Adam(generator.parameters(), lr=2e-4, betas = (0.5, 0.999))

    for i in range(len(input_imgs)):
        real_img, input_img = real_imgs[i], input_imgs[i]
        x_generator[i, :, :, :] = input_img
        
    generator_out = generator.forward(x_generator)

    x_discriminator = torch.cat((x_generator, generator_out), dim=0)

    # Right Column
    discriminator_out = discriminator.forward(x_discriminator)

    loss_1 = loss_L1(generator_out, real_img)
    loss_2 = loss_BCE(discriminator_out, torch.ones(30, 30))

    total_loss = 100*loss_1 + loss_2

    #Apply Gradients
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()

    return total_loss.item()


folder_dir_gt = folder_dir_in_matthijs
folder_dir_in = folder_dir_gt_matthijs

in_list = [os.path.join(folder_dir_in, filename) for filename in os.listdir(folder_dir_in)]
gt_list = [os.path.join(folder_dir_gt, filename) for filename in os.listdir(folder_dir_gt)]
enhanced_list = [os.path.join(folder_dir_in, filename) for filename in os.listdir(folder_dir_in)]

In [ ]:
train_discriminator()

In [ ]:
class PairedImageDataset(Dataset):
    def __init__(self, rootA, rootB, transform=None):
        """
        Args:
            rootA (string): Directory with all the images in trainA.
            rootB (string): Directory with all the images in trainB.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.rootA = rootA
        self.rootB = rootB
        self.transform = transform

        # Assuming filenames in both folders are the same and in order
        self.filenames = sorted(os.listdir(rootA))

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        img_nameA = os.path.join(self.rootA, self.filenames[idx])
        imageA = Image.open(img_nameA).convert('RGB')

        img_nameB = os.path.join(self.rootB, self.filenames[idx])
        imageB = Image.open(img_nameB).convert('RGB')

        if self.transform:
            imageA = self.transform(imageA)
            imageB = self.transform(imageB)

        return imageA, imageB

def try_gpu():
    """
    If GPU is available, return torch.device as cuda:0; else return torch.device
    as cpu.
    """
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu')
    return device




class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        # test
        dim_in_encoder = [3, 64, 128, 256, 512, 512, 512, 512, 512, 512]
        dim_in_decoder = [512, 512, 512, 512, 512, 256, 128, 64, 3]

        self.layers = nn.ModuleList()

        ####################################################################################
        # encoder
        ####################################################################################

        # layer 1
        self.layers.append(
            nn.Conv2d(
                in_channels=dim_in_encoder[0],
                out_channels=dim_in_encoder[1],
                kernel_size=(4, 4),
                stride=2,
                padding=1,
            )
        )

        self.layers.append(nn.LeakyReLU(0.2))

        # the rest of the layers
        for in_out in range(1, (len(dim_in_encoder) - 1)):
            self.layers.append(
                nn.Conv2d(
                    in_channels=dim_in_encoder[in_out],
                    out_channels=dim_in_encoder[(in_out + 1)],
                    kernel_size=(4, 4),
                    stride=2,
                    padding=1,
                )
            )

            self.layers.append(nn.BatchNorm2d(dim_in_encoder[in_out + 1]))

            self.layers.append(nn.LeakyReLU(0.2))

        print("layer", len(self.layers))

        ####################################################################################
        # decoder
        ####################################################################################

        # layers 1 to 3 with dropout
        for in_out in range(0, 3):
            self.layers.append(
                nn.ConvTranspose2d(
                    in_channels=dim_in_decoder[in_out],
                    out_channels=dim_in_decoder[(in_out + 1)],
                    kernel_size=(4, 4),
                    stride=2,
                    padding=1,
                )
            )

            self.layers.append(nn.BatchNorm2d(dim_in_decoder[in_out + 1]))
            self.layers.append(nn.Dropout2d(0.5))
            self.layers.append(nn.LeakyReLU(0.2))

        # the rest of the layers without dropout
        for in_out in range(3, (len(dim_in_decoder) - 1)):
            self.layers.append(
                nn.ConvTranspose2d(
                    in_channels=dim_in_decoder[in_out],
                    out_channels=dim_in_decoder[(in_out + 1)],
                    kernel_size=(4, 4),
                    stride=2,
                    padding=1,
                )
            )

            self.layers.append(nn.BatchNorm2d(dim_in_decoder[in_out + 1]))
            self.layers.append(nn.Dropout2d(0)) #todo()
            self.layers.append(nn.LeakyReLU(0.2))

        print("layer", len(self.layers))

    def forward(self, x):
        encoder_out = []

        #forward on first layer
        x = self.forward_2_lay(x, 0)
        encoder_out.append(x)
        # print("first pass", x.shape)

        #forward encoder and save layers
        for i in range(0, 6): # 1 - 8
            x = self.forward_3_lay(x, i, 2)
            encoder_out.append(x)
            # print("second loop", x.shape)

        #Do layer 512x1x1
        x = self.forward_3_lay(x, 6, 2)
        #Do layer 512x2x2
        x = self.forward_4_lay(x, 0, 26)

        #Do forward on decoder
        for i in range(0, 7):
            # print("encoder", encoder_out[6 - i].shape,"x", x.shape)
            x = self.forward_4_lay(torch.add(x, encoder_out[6 - i]), i, 30)
            # print("decoder for loop", x.shape)

        x = torch.tanh(x)
            
        return x

    def forward_2_lay(self, x, layer_num):

        for i in range(0, 2):
            x = self.layers[i + 2 * layer_num](x)

        return x

    def forward_3_lay(self, x, layer_count, start):

        for i in range(0, 3):
            x = self.layers[i + 3 * layer_count + start](x)

        return x

    def forward_4_lay(self, x, layer_num, start):
        # print(len(self.layers))
        for i in range(0, 4):
            # print(i + 4 * layer_num)
            x = self.layers[i + 4 * layer_num + start](x)

        return x



def train(train_loader, net, optimizer, criterion, device='cpu'):
    """
    Trains network for one epoch in batches.

    Args:
        train_loader: Data loader for training set.
        net: Neural network model.
        optimizer: Optimizer (e.g. SGD).
        criterion: Loss function (e.g. cross-entropy loss).
        device: whether the network runs on cpu or gpu
    """

    avg_loss = 0

    # iterate through batches
    for i, data in enumerate(train_loader):
        # get the inputs; data is a list of [inputs, labels]
        inputs, good_images = data

        # convert the inputs to run on GPU if set
        inputs, good_images = inputs.to(device), good_images.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)

        loss = criterion(outputs, good_images) # compare output with labels
        loss.backward()
        optimizer.step()

        # keep track of loss and accuracy
        avg_loss += loss

    return avg_loss / len(train_loader)

def run_saved():
    # Define your transformations
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
    ])

    # Initialize the dataset
    paired_dataset_dark = PairedImageDataset(rootA=r'/kaggle/input/euvp-dataset/EUVP/Paired/underwater_dark/trainA',
                                        rootB=r'/kaggle/input/euvp-dataset/EUVP/Paired/underwater_dark/trainB',
                                        transform=transform)

    # Create instance of Autoencoder
    device = try_gpu()

    model = torch.load('model.pth')
#     model_saved.load('saved_model_full.pth')
     # Load state dictionary  # make a folder
    
    count = 0
    image_counter = 10
    path = 'saved'
    path_loop = None
    # Initialize DataLoader
    EUVP_data = DataLoader(paired_dataset_dark, batch_size=5, shuffle=False, num_workers=16)
    print("start printing")
    
    path_ImageA = f"ImageA"
    path_ImageB = f"ImageB"
    path_ImageG = f"ImageG"
    
    if not (os.path.exists(path_loop) and os.path.isdir(path_loop)):
        os.makedirs(path_loop)
    
    # Store somep images
    for i, data in enumerate(EUVP_data):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        y_out = model_saved.forward(inputs)

        for x in range(len(inputs)):
            name = (f'{path_ImageA}/image{count}.jpg')
            save_image(inputs[x], name)
            name = (f'{path_ImageB}/image{count}.jpg')
            save_image(y_out[x], name)
            name = (f'{path_ImageG}/image{count}.jpg')
            save_image(labels[x], name)
            count += 1

            if (count == image_counter):
                break
        break

def main():
    torch.cuda.empty_cache()
    # Define your transformations
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
    ])

    # Initialize the dataset
    paired_dataset_dark = PairedImageDataset(rootA=r'/kaggle/input/euvp-dataset/EUVP/Paired/underwater_dark/trainA',
                                        rootB=r'/kaggle/input/euvp-dataset/EUVP/Paired/underwater_dark/trainB',
                                        transform=transform)
    # Initialize the dataset
    paired_dataset_imagenet = PairedImageDataset(rootA=r'/kaggle/input/euvp-dataset/EUVP/Paired/underwater_imagenet/trainA',
                                        rootB=r'/kaggle/input/euvp-dataset/EUVP/Paired/underwater_imagenet/trainB',
                                        transform=transform)

    # Initialize the dataset
    paired_dataset_scenes = PairedImageDataset(rootA=r'/kaggle/input/euvp-dataset/EUVP/Paired/underwater_scenes/trainA',
                                        rootB=r'/kaggle/input/euvp-dataset/EUVP/Paired/underwater_scenes/trainB',
                                        transform=transform)
    
    # Initialize DataLoader
    EUVP_data = DataLoader(paired_dataset_dark, batch_size=360, shuffle=True, num_workers=8)
    
        # Initialize DataLoader
    EUVP_data1 = DataLoader(paired_dataset_imagenet, batch_size=360, shuffle=True, num_workers=8)
    
        # Initialize DataLoader
    EUVP_data2 = DataLoader(paired_dataset_scenes, batch_size=360, shuffle=True, num_workers=8)

    # initialize model
    model = Generator()

    # Create a writer to write to Tensorboard
    writer = SummaryWriter()

    # Create instance of Autoencoder
    device = try_gpu()
    print(device)
    if torch.cuda.is_available():
        model = nn.DataParallel(model)
        model.cuda()
    # AE2 = Generator(latent_dims[0], s_img, hdim = hdim).to(device) #2-dimensional latent space
    # AE3 = Generator(latent_dims[1], s_img, hdim = hdim).to(device) #3-dimensional latent space

    # Create loss function and optimizer
    criterion = F.mse_loss

    optimizer = optim.Adam(model.parameters(), lr=5e-4)

    # Set the number of epochs to for training
    epochs = 20

   
    for epoch in tqdm(range(epochs)):  # loop over the dataset multiple times
        # Train on data
        train_loss = train_generator()
        # Write metrics to Tensorboard
        writer.add_scalars("Loss", {'Train': train_loss}, epoch)
        
        # Train on data
        train_loss = train(EUVP_data1, model, optimizer, criterion, device)
        # Write metrics to Tensorboard
        writer.add_scalars("Loss", {'Train': train_loss}, epoch)
        
        # Train on data
        train_loss = train(EUVP_data2, model, optimizer, criterion, device)
        # Write metrics to Tensorboard
        writer.add_scalars("Loss", {'Train': train_loss}, epoch)
    
    # save
    torch.save(model, 'model.pth')
    torch.save(model.state_dict(), 'model_dict.pth')
    
    torch.cuda.empty_cache()